In [1]:
import os
import re
from collections import defaultdict

# Xilinx - sch2vf(一般sch都有.vf文件, 该函数可生成sch缺失的.vf文件)

In [3]:
def sch2verilog(path):
    sch_fns = [f for f in os.listdir(path) if f.split('.')[-1] == 'sch'] 
    vfns = [fn.split('.')[0] for fn in sch_fns if not os.path.exists(os.path.join(path,fn.split('.')[0]+'.vf'))]
    ttl = len(sch_fns)
    pat = 'No Error found'
    mat = re.compile(pat)
    fail = 0
    for i,fn in enumerate(vfns):
        #cmd_seg = ['sch2hdl -intstyle ise -family aspartan6 -verilog ']
        cmd_seg = ['sch2hdl -intstyle ise -family spartan6 -verilog ']
        vfn = [os.path.join(path,fn)]
        cmd = ''.join(cmd_seg + vfn + ['.vf'] + [' -w '] + vfn + ['.sch'])
        rtn = None
        with os.popen(cmd,'r') as f:
            rtn = f.read() 
        if not mat.search(rtn):
            fail += 1
        print('\rSCH2HDL: {}\{}, Fail: {}'.format(i+1,ttl,fail),end='')
    print()
    print('**Process Finished**')

In [4]:
def factory_set(gpath,dpath):
    gfns = [fn for fn in os.listdir(gpath) if fn.split('.')[-1] == 'vf']
    dfns = [fn for fn in os.listdir(dpath) if fn.split('.')[-1] == 'vf']
    rb_fns = list(set(dfns) - set(gfns))
    print('Del Files In: {}'.format(dpath))
    print(len(gfns),len(dfns),len(rb_fns))
    #cmd = input('Yes/NO?')
    cmd = 'Y'
    if cmd == 'Y' or 'y':
        print()
        del_fns = 0
        for fn in rb_fns: 
            fp = os.path.join(dpath,fn)
            #print(fp)
            if os.path.exists(fp):
                os.remove(fp)
                del_fns += 1
                #print('fn')
                print('\r**Del Files: {}/{}**'.format(del_fns,len(rb_fns)),end='')
        print()
        print('**Del Done**')        

In [347]:
factory_set(gold_path,src_path)

Del Files In: e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/
51 51 0


**Del Done**


In [308]:
os.remove(os.path.join(dd_p,os.listdir(dd_p)[0]))

# ########################

# 1. 设置输入输出目录

In [2]:
#src_path = 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28'
#top_vf = os.path.join(src_path,'a_vcm_pro.vf')
#top_prj = os.path.join(src_path,'a_vcm_pro.prj')
#dst_path = 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325'
#src_path = 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/vcar/Mini_Rcv_V-Car1026/'
#dst_path = 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/Mini_Rcv_20220714/'
#top_vf = os.path.join(src_path,'rcv.vf')
#top_prj = os.path.join(src_path,'rcv.prj')
#src_path = 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/'
#dst_path = 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220715/'
gold_path = 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/raw/Mini_rcv_CPNdual_C_All/'
src_path = 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/'
dst_path = 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/'
if not os.path.exists(dst_path):
    os.makedirs(dst_path)

# 2. 从.xise中获取Xilinx工程模块清单

In [5]:
def sub_mdl_sp(spath):
    pat = r'"Output File Name" xil_pn:value="(.+?)"'
    mat = re.compile(pat)
    xise_fp = [os.path.join(spath,fn) for fn in os.listdir(spath) if os.path.splitext(fn)[-1] == '.xise'][0]
    print('XISE: ',xise_fp)
    data = None
    with open(xise_fp,'r') as f:
        data = f.read()
    prj_fsn = mat.findall(data)[0]
    fpath = os.path.join(spath,prj_fsn+'.prj')
    ftbe = os.path.exists(fpath)
    print('PRJ: {} - Real: {}'.format(fpath,ftbe))
    data = None
    with open(fpath,'r') as f:
        data = f.read()    
    mdl_ns = []   
    #fon = os.path.splitext(fpath)[0]
    fdix = os.path.splitext(fpath)[-1]
    #fpv = fon + '.v'
    #fvf = fon + '.vf'
    p = None
    if fdix == '.prj':       #Top
        p = r'"(.+)"'
        m = re.compile(p)
        mdl_ns = m.findall(data)
    return mdl_ns

In [6]:
sub_mdl_sp(src_path)

XISE:  e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/rcv.xise
PRJ: e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/rcv.prj - Real: True


['ipcore_dir/XSPx72M8WE.v',
 'tx_test.v',
 'sens_crc.v',
 'sel8.vf',
 'rtn_mux_b.v',
 'rgmii_tx.vf',
 'ipcore_dir/dist_32x16m.v',
 'fd8.vf',
 'fd1.vf',
 'ctrl_data_plus.v',
 'tx_process.vf',
 'signed_mult_18x18.v',
 'sel16.vf',
 'sdp_bram_a8b_d8b.v',
 'sdp_bram_a10b_d30b.v',
 'sdp_bram_a10b_d16b.v',
 'pti_source/MRCV_PLL.v',
 'pti_source/MIN_CPNL_RDCNT.v',
 'pti_source/MIN_CPDT_BUF.v',
 'pti_source/MIN_BRTC_BF.v',
 'pti_source/MCU_UART_IF.v',
 'pti_source/GOE_SYNC_RSTN.v',
 'pti_source/FRMM_SDABIT.v',
 'pti_source/FRMM_MENCT.v',
 'pti_source/DR8W16_32W4R.v',
 'pti_source/D32W2_8W64R.v',
 'pti_source/D32IN_8BOUT_BUF.v',
 'pti_source/CRC32_ETHER.v',
 'pti_source/CLOCK_CONVERT_BUF.v',
 'pac_stp_gen.v',
 'ovp_gen_a.v',
 'm8_1e16b.vf',
 'm4_postmult_xt2p5_a.v',
 'ipcore_dir/XSP9x512WE.v',
 'ipcore_dir/XSP32x1024M4WE.v',
 'gamma30_rom_mpc.v',
 'gamma28_rom_mpc.v',
 'gamma25_rom_mpc.v',
 'gamma24_rom_mpc.v',
 'gamma22_rom_mpc.v',
 'gamma20_rom_mpc.v',
 'gamma18_rom_mpc.v',
 'gamma10_rom_mpc.v

In [7]:
def halo(fdat,mdl_ns,dpath,update=True):

    gen_sub_mdls = []
    
    m0 = re.compile(r'(module.+?endmodule)',re.DOTALL) 
    elms = m0.findall(fdat)
    #print(len(elms))
    m1 = re.compile(r'_MUSER\w+|_HXILINX\w+')
    m2 = re.compile(r'module (\w+)')
    #elms_new = []
    for elm in elms:
        elm_d = m1.sub('',elm)
        #print(elm_d)
        #elm_n = m2.search(elm_d).span()[0]
        elm_n = m2.findall(elm_d)[0]
        #print(elm_n)
        elm_fn = elm_n + '.v'
        if elm_fn not in mdl_ns:
            opath = os.path.join(dpath,elm_fn)
            gen_sub_mdls.append(opath)

            if not os.path.exists(opath) or update:
                with open(opath,'w') as f:
                    f.write(elm_d)
                print('**VLG: {} Generated Done**'.format(elm_fn))             
    return gen_sub_mdls
    

In [8]:
#def vf_rush(vf_fn,mdl_ns,spath,dpath,update=False):
#def fprj_flu(mdl_ns,spath,dpath,update=False,x2x=True):
def fprj_flu(mdl_ns,spath,dpath,update=False,targe_family=1):

    x2x = 0    
    if targe_family == 0:
        x2x = 1
    else:
        x2x = 0
    #UCF
    if x2x:
        ucf_fps = [os.path.join(spath,fn) for fn in os.listdir(spath) if os.path.splitext(fn)[-1] == '.ucf']
        for fp in ucf_fps:
            data = None
            fn = os.path.split(fp)[-1]
            dfp = os.path.join(dpath,fn)
            if os.path.exists(fp): 
                if not os.path.exists(dfp) or update:
                    with open(fp,'rb') as f:
                        data = f.read()
                        with open(dfp,'wb') as ff:
                            ff.write(data)
                    print('**Copyed {} Done**'.format(fn))                
                else:
                    print('**File: {} NOP**'.format(fn))                
   
    ip_fpaths = []
    leaf_fpaths = []

    #IP&Module
    for mdl_n in mdl_ns:
        opath = os.path.splitext(os.path.join(dpath,mdl_n))[0] + '.v'
        
        ipath = os.path.join(spath,mdl_n)
        fname, ftype = os.path.split(ipath)[-1].split('.')
        
        ip_dir = os.path.join(spath,'ipcore_dir')
        
        ift = os.path.join(ip_dir,fname)
        ift_a = os.path.join(spath,fname)
        oft = os.path.splitext(opath)[0]
        
        i_ngc = ift + '.ngc'
        i_ngc_a = ift_a + '.ngc'
        o_ngc = oft + '.ngc'
        i_coe = ift + '.coe'
        i_coe_a = ift_a + '.coe'
        o_coe = oft + '.coe'
        i_mif = ift + '.mif'
        i_mif_a = ift_a + '.mif'
        o_mif = oft + '.mif'    
        
        vname = fname + '.v'
        ngc_n = fname + '.ngc'
        coe_n = fname + '.coe'
        mif_n = fname + '.mif'    
        #vname = os.path.split(opath)[-1]    
        #print(opath)           
        
        isIP = False
        #ip_fp = [] 
        
        #if os.path.exists(i_ngc):
        path = ''
        if os.path.exists(i_ngc_a):
            path = i_ngc_a
        elif os.path.exists(i_ngc):
            path = i_ngc
        if path:
            isIP = True
            #ip_fp.append(path)
            if x2x:
                if not os.path.exists(o_ngc) or update:
                    with open(path,'rb') as f:
                        data = f.read()        
                        with open(o_ngc,'wb') as ff:
                            ff.write(data) 
                    print('**NGC: {} Copyed Done**'.format(ngc_n))
            
        #path = ''
        if os.path.exists(i_mif_a):
            path = i_mif_a
        elif os.path.exists(i_mif):
            path = i_mif
        if path:
            #ip_fp.append(path)
            if x2x:
                if not os.path.exists(o_mif) or update:               
                    with open(path,'rb') as f:
                        data = f.read()       
                        with open(o_mif,'wb') as ff:
                            ff.write(data) 
                    print('**MIF: {} Copyed Done**'.format(mif_n))
            
        #print(update,os.path.exists(opath))
        odir = os.path.split(opath)[0]    
        #if not os.path.exists(odir) and x2x:
        if x2x or not isIP and not os.path.exists(odir):
            os.makedirs(odir)        
        if ftype == 'vf':
            with open(ipath,'r') as f:
                data = f.read()
            gen_mdls = halo(data,mdl_ns,dpath,update=update)
            leaf_fpaths += gen_mdls
        elif ftype == 'v':
            if isIP and x2x or not isIP:
                if not os.path.exists(opath) or update:
                    with open(ipath,'rb') as f:
                        data = f.read()        
                    with open(opath,'wb') as f:
                        f.write(data)
                    print('**VLG: {} Copyed Done**'.format(vname))
            if isIP:
                #ip_fp.append(ipath)
                ip_fpaths.append(ipath)
            else:
                leaf_fpaths.append(opath)
        #else:
        #    print('**File: {} No Change**'.format(vname)) 
        #if len(ip_fp) > 1:
        #    ip_fpaths.append(ip_fp)
    return ip_fpaths, leaf_fpaths

In [54]:
vlg_core_path = 'D:/software/Xilinx/14.7/ISE_DS/ISE/verilog/src/XilinxCoreLib/'
vdl_core_path = 'D:/software/Xilinx/14.7/ISE_DS/ISE/vhdl/src/XilinxCoreLib/'

In [56]:
vlg_hc_mns = [fn.split('.')[0] for fn in os.listdir(vlg_core_path)]
vdl_hc_mns = [fn.split('.')[0] for fn in os.listdir(vdl_core_path)] 

In [57]:
len(vlg_hc_mns),len(vdl_hc_mns)

(200, 1176)

In [59]:
len(set(vlg_hc_mns) & set(vdl_hc_mns)), len(set(vdl_hc_mns) - set(vlg_hc_mns))

(40, 1136)

In [60]:
pt1 = r'(\w+) #'
mh1 = re.compile(pt1)

In [67]:
mh1.search(ngc_dat)[1]

'BLK_MEM_GEN_V7_3'

In [73]:
src_prj = os.path.join(dst_path,'impl\gwsynthesis')

In [74]:
os.listdir(src_prj)

['min_dual_rcv.log', 'min_dual_rcv.prj']

In [9]:
def get_gw_FPGA_info(prj_path):
    gws_path = os.path.join(prj_path,'impl\gwsynthesis')
    fprj = [os.path.join(gws_path,fn) for fn in os.listdir(gws_path) if fn.split('.')[1] == 'prj'][0]
    print('Gowin Synthesis File: {}'.format(fprj))
    pt = r'Device id="(.+)" package="(.+)" speed="\w+" partNumber="(.+)"'
    mh = re.compile(pt)
    data = None
    res = []
    for line in open(fprj,'r'):
        res = mh.findall(line)
        if res:
            break
    #print(res[0])
    dev_info = []
    dev_info.append(res[0][0].split('-')[0])
    dev_info += list(res[0])
    return dev_info

In [10]:
get_gw_FPGA_info(dst_path)

Gowin Synthesis File: e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/impl\gwsynthesis\min_dual_rcv.prj


['GW2A', 'GW2A-55', 'UBGA324D', 'GW2A-LV55UG324DC7/I6']

In [11]:
def gw_mod_attrs_rtn(xip_type):
    sample = ''
    if xip_type == 'bram_sdp': 
        sample = './mod/bram/bram_sdp/bram_sdp.mod'
    elif xip_type == 'prom': 
        sample = './mod/bram/prom/prom.mod'
    elif xip_type == 'dram_sdp': 
        sample = './mod/dram/dram_sdp/dram_sdp.mod'
    elif xip_type == 'sign_mult': 
        sample = './mod/sign_mult/sign_mult.mod'
    data = []
    with open(sample,'r') as f:
        data = f.readlines()
    #print(data)
    params = [x.split()[0] for x in data if x.strip()]
    #bram_attr = {x.split()[0]: '' for x in data  if x.strip()}
    return params

In [12]:
gw_mod_attrs_rtn('sign_mult')

['-series',
 '-device',
 '-package',
 '-part_number',
 '-mod_name',
 '-file_name',
 '-path',
 '-type',
 '-file_type',
 '-source_a',
 '-source_b',
 '-width_a',
 '-width_b',
 '-data_type_a',
 '-data_type_b',
 '-en_shiftout_a',
 '-en_shiftout_b',
 '-reset_mode',
 '-ina_reg',
 '-inb_reg',
 '-pipe_reg',
 '-out_reg']

In [13]:
def gw_core_gen(mod_fp): 
    m1 = re.compile(r'Error')    
    m2 = re.compile(r'Successfully')
    m3 = re.compile(r'MG2116')
    m4 = re.compile(r'MG2001')
    ip_name = os.path.split(mod_fp)[-1].split('.')[0]
    cmd = 'GowinModGen -do ' + mod_fp
    #rtn = os.popen(cmd,'r').readlines()
    rtn = None
    with os.popen(cmd,'r') as f:
        rtn = f.read()
    r1 = m1.search(rtn)
    r2 = m2.search(rtn)
    e1 = m3.search(rtn)
    e2 = m4.search(rtn)
    #print(type(rtn))
    #print(rtn)
    #print(r1,r2,e1)
    if r1:
        if e1:
            print(' ** Failed: Path Error - {} **'.format(ip_name))
        elif e2:
            print(' ** Failed: Type Error - {} **'.format(ip_name))
    elif r2:
        print(' ** {}: Done **'.format(ip_name))
    else:
        print(' ** Failed: Port Wdith Error - {} **'.format(ip_name))

In [14]:
gw_core_gen('E:/mlfield/xex/py/mod/bram/prom/prom.mod')

In [14]:
def xlx_IP_miner(spath,dpath,target_family=1):
    p1 = r'module (.+)'
    m1 = re.compile(p1)
    p2 = r'(.+) #'
    m2 = re.compile(p2)    
    fname = os.path.split(spath)[-1].split('.')[0]
    sdir = os.path.split(spath)[0]
    ip_dir = os.path.join(sdir,'ipcore_dir')
    print(' ..Processing: ',fname)
    mdl_name = ''
    xip_type = ''
    
    mif_in_sp = os.path.splitext(spath)[0] + '.mif'
    mif_in_ip = os.path.join(ip_dir, fname+'.mif')
    mif_spath = ''
    if os.path.exists(mif_in_sp):
        mif_spath = mif_in_sp
    elif os.path.exists(mif_in_ip):
        mif_spath = mif_in_ip
    mi_path = ''

    new_path = os.path.join(dpath,'ip/'+fname+'/')
    if mif_spath:
        if target_family == 1:
            mi_path = os.path.join(new_path,fname+'.mi')
        #print(' ..mif_path: ',mif_spath)
        #print(' ..mi_path: ',mi_path)
    #print('gen_path: ',new_path)
    if not os.path.exists(new_path):
        os.makedirs(new_path)    
    
    ip_type = '' 
    ip_parms = []
    
    paras = []
    ip_dat = None
    with open(spath,'r') as f:
        ip_dat = f.read()

    res = m1.search(ip_dat)
    r1 = res[1]
    mdl_st = res.span()[0]
    if r1:
        mdl_name = r1.strip('(').strip()
        #print('mod_name: ',mdl_name)
    r2 = m2.search(ip_dat[mdl_st:])[1].strip()
    if r2:
        xip_type = r2
        #print('type: ',xip_type) 
    if 'BLK_MEM' in xip_type:
        ip_type, ip_parms = xlx_RAM_proc(ip_dat,0,mif_spath=mif_spath,mi_dpath=mi_path)
    elif 'DIST_MEM' in xip_type:
        ip_type, ip_parms = xlx_RAM_proc(ip_dat,1,mif_spath=mif_spath,mi_dpath=mi_path)
    elif 'DSP48' in xip_type:
        #print('DSP')
        ip_type, ip_parms = xlx_DSP_proc(ip_dat)
    paras = [mdl_name, fname, new_path] + ip_parms
    #print('  ..IP_type: ', ip_type)
    return ip_type, paras        

In [15]:
'SYNC'.lower()

'sync'

In [15]:
def xlx_RAM_proc(ip_dat, mem_type, mif_spath='', mi_dpath='', target_family=1):
    p1 = r'C_MEM_TYPE\((\d)\)'
    m1 = re.compile(p1)
    p2 = r'C_READ_DEPTH_B\((\d+)\)'
    m2 = re.compile(p2)
    p3 = r'C_READ_WIDTH_B\((\d+)\)'
    m3 = re.compile(p3)
    p4 = r'C_RST_TYPE\("(\w+)"\)'
    m4 = re.compile(p4)
    p5 = r'C_WRITE_DEPTH_A\((\d+)\)'
    m5 = re.compile(p5)
    p6 = r'C_WRITE_WIDTH_A\((\d+)\)'
    m6 = re.compile(p6)
    p7 = r'C_DEPTH\((\d+)\)'
    m7 = re.compile(p7)
    p8 = r'C_WIDTH\((\d+)\)'
    m8 = re.compile(p8)    
    
    xip_type = ''
    paras = []
       
    c_mem_type = int(m1.search(ip_dat)[1])

    if mem_type == 0 and c_mem_type == 1:    #BRAM_SDP
        c_read_dpth_B  = m2.search(ip_dat)[1]
        c_read_wdth_B  = m3.search(ip_dat)[1]
        c_rst_type     = m4.search(ip_dat)[1]
        c_write_dpth_A = m5.search(ip_dat)[1]
        c_write_wdth_A = m6.search(ip_dat)[1]  
        
        if target_family == 1:
            xip_type = 'bram_sdp'
            paras += ['RAM_SDP', 
                      'vlg',
                      'true',
                      c_read_dpth_B,
                      c_read_wdth_B,
                      c_write_dpth_A,
                      c_write_wdth_A,
                      'bypass',
                      c_rst_type.lower()]
        
    elif mem_type == 0 and c_mem_type == 3: #BROM_SP
        c_read_dpth_B  = m2.search(ip_dat)[1]
        c_read_wdth_B  = m3.search(ip_dat)[1]
        c_rst_type     = m4.search(ip_dat)[1]
        
        if target_family == 1:
            xip_type = 'prom'
            if mif_spath:
                gowin_mi_gen(mif_spath,mi_dpath)
            #print(mif_spath,mi_dpath)
            paras += ['RAM_ROM', 
                      'vlg',
                      'true',
                      c_read_dpth_B,
                      c_read_wdth_B,
                      'bypass',
                      c_rst_type.lower(),
                     mi_dpath] 

    elif mem_type == 1 and c_mem_type == 4: #DRAM_SDP
        c_dpth  = m7.search(ip_dat)[1]
        c_wdth  = m8.search(ip_dat)[1]
        
        if target_family == 1:
            xip_type = 'dram_sdp'
            paras += ['SSRAM_SDP', 
                      'vlg',
                      c_dpth,
                      c_wdth] 
        
    return xip_type, paras

In [16]:
def xlx_DSP_proc(ip_dat, target_family=1):
    p1 = r'A0REG \( (\d) \)'
    m1 = re.compile(p1)
    p2 = r'A1REG \( (\d) \)'
    m2 = re.compile(p2)
    p3 = r'B0REG \( (\d) \)'
    m3 = re.compile(p3)
    p4 = r'B1REG \( (\d) \)'
    m4 = re.compile(p4)
    p5 = r'MREG \( (\d) \)'
    m5 = re.compile(p5) 
    p6 = r'PREG \( (\d) \)'
    m6 = re.compile(p6)    
    p7 = r'RSTTYPE \( "(\w+)" \)'
    m7 = re.compile(p7)
    
    paras = []
    
    if target_family == 1:
        in_a_registered = int(m1.search(ip_dat)[1])
        pl_a_registered = int(m2.search(ip_dat)[1])    
        in_b_registered = int(m3.search(ip_dat)[1])
        pl_b_registered = int(m4.search(ip_dat)[1])    
        ot_m_registered = int(m5.search(ip_dat)[1]) 
        ot_p_registered = int(m6.search(ip_dat)[1]) 
        rst_type = m7.search(ip_dat)[1]
        
        paras = ['MULT', 
                 'vlg', 
                 'parallel', 
                 'parallel',
                 '18',
                 '18',
                 'signed',
                 'signed',
                 'false',
                 'false',
                 rst_type.lower(),
                 'registered' if in_a_registered else 'bypass',
                 'registered' if in_b_registered else 'bypass',
                 'registered' if pl_a_registered and pl_b_registered else 'bypass',
                 'registered' if ot_m_registered or ot_p_registered else 'bypass']
        
    return 'sign_mult', paras

In [17]:
def gowin_mi_gen(src_path,gw_path):
    mif_data = None
    with open(src_path,'r',encoding='utf-8') as f:
        mif_data = f.readlines()
    adr_dpth = '#Address_depth=' + str(len(mif_data)) + '\n'
    dat_wdth = '#Data_width=' +  str(len(mif_data[0].strip())) + '\n'
    head = ['#File_format=Bin\n', adr_dpth, dat_wdth]
    new_mif = head + mif_data
    with open(gw_path,'w',encoding='utf-8') as f:
        f.writelines(new_mif)

In [18]:
def x2g_ip_regen(dev_info, xip_type, gw_mod_attrs, gw_ip_paras, update=True):
    #xip_type, gw_ip_paras = xlx_IP_miner(xip_path,dpath)
    gw_dev_info = None
            
    vlg_path = os.path.join(gw_ip_paras[2], gw_ip_paras[1])
    mdl_name = gw_ip_paras[0]
    
    if update:
        if isinstance(dev_info,list):
            gw_dev_info = dev_info.copy()
        mod_new_dir = gw_ip_paras[2]
        mod_new_fn = gw_ip_paras[1] + '.mod'
        mod_new_fpath = os.path.join(mod_new_dir,mod_new_fn)
        #print('mod_fpath: ',mod_new_fpath)
        if xip_type == 'dram_sdp':
            gw_dev_info.append(gw_dev_info[1])    
        gw_mod_valus = gw_dev_info + gw_ip_paras
        #gw_mod_attrs = gw_mod_attrs_rtn(xip_type)
        gw_mod_res = [gw_mod_attrs[i] + ' ' + gw_mod_valus[i] + '\n' for i in range(len(gw_mod_attrs))]
        #print('mod: ',gw_mod_res)
        with open(mod_new_fpath,'w') as f:
            f.writelines(gw_mod_res)
        #print(' ..Mod File Generated')
        gw_core_gen(mod_new_fpath)
    
    return mdl_name, vlg_path

In [19]:
def x2g_IPs_proc(xip_fps, dpath, update=True):
    gw_mod_attr_books = {}
    gw_dev_info = None
    if update:
        gw_dev_info = get_gw_FPGA_info(dpath)
    xps = None
    if isinstance(xip_fps, list):
        xps = xip_fps
    elif isinstance(xip_fps, str):
        xps = [xip_fps]

    #ip_names = []
    #ip_fpaths = []
    gw_ips_lib = {}
    gw_tmp_lib = {}
    
    for fp in xps:
        ip_type, gw_ip_paras = xlx_IP_miner(fp, dpath)
        if ip_type not in gw_mod_attr_books:
            gw_mod_attr_books[ip_type] = gw_mod_attrs_rtn(ip_type)
        gw_mod_attrs = gw_mod_attr_books[ip_type]
        ip_name, ip_fpath = x2g_ip_regen(gw_dev_info, ip_type, gw_mod_attrs, gw_ip_paras, update=update)
        #tmp_fpath = ip_fpath + '_tmp.v'
        if ip_name:
            if ip_name not in gw_ips_lib:
                gw_ips_lib[ip_name] = {}
                if ip_type not in gw_tmp_lib:
                    gw_tmp_lib[ip_type] = gw_tmp_extractor(ip_fpath)
                gw_ips_lib[ip_name]['type'] = ip_type
                gw_ips_lib[ip_name]['fpath'] = ip_fpath + '.v'
                gw_ips_lib[ip_name]['tmp'] = gw_tmp_lib[ip_type]
        print('  ..Done..  ')
        #ip_names.append(ip_name)
        #ip_fpaths.append(ip_fpath)
    #return ip_names, ip_fpaths
    return gw_ips_lib

In [221]:
xip_sdp_fp = xip_fps[4]

In [222]:
xip_sdp_fp

'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/sdp_bram_a10b_d30b.v'

In [223]:
sdp_eg_dpath = './mod/'

In [333]:
x2g_IPs_proc(xip_sdp_fp,dst_path,update=False)

 ..Processing:  sdp_bram_a10b_d30b
  ..Done..  


{'sdp_bram_a10b_d30b': {'type': 'bram_sdp',
  'fpath': 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/ip/sdp_bram_a10b_d30b/sdp_bram_a10b_d30b.v',
  'tmp': {'1st': '    sdp_bram_a10b_d30b',
   'dout': '        .dout(dout_o), \n',
   'clka': '        .clka(clka_i), \n',
   'cea': '        .cea(cea_i), \n',
   'reseta': '        .reseta(reseta_i), \n',
   'clkb': '        .clkb(clkb_i), \n',
   'ceb': '        .ceb(ceb_i), \n',
   'resetb': '        .resetb(resetb_i), \n',
   'oce': '        .oce(oce_i), \n',
   'ada': '        .ada(ada_i), \n',
   'din': '        .din(din_i), \n',
   'adb': '        .adb(adb_i) \n',
   '-1l': '    );\n'}}}

In [296]:
x2g_IPs_proc(xip_fps,dst_path,update=False)

 ..Processing:  XSPx72M8WE
  ..Done..  
 ..Processing:  dist_32x16m
  ..Done..  
 ..Processing:  signed_mult_18x18
  ..Done..  
 ..Processing:  sdp_bram_a8b_d8b
  ..Done..  
 ..Processing:  sdp_bram_a10b_d30b
  ..Done..  
 ..Processing:  sdp_bram_a10b_d16b
  ..Done..  
 ..Processing:  XSP9x512WE
  ..Done..  
 ..Processing:  XSP32x1024M4WE
  ..Done..  
 ..Processing:  gamma30_rom_mpc
  ..Done..  
 ..Processing:  gamma28_rom_mpc
  ..Done..  
 ..Processing:  gamma25_rom_mpc
  ..Done..  
 ..Processing:  gamma24_rom_mpc
  ..Done..  
 ..Processing:  gamma22_rom_mpc
  ..Done..  
 ..Processing:  gamma20_rom_mpc
  ..Done..  
 ..Processing:  gamma18_rom_mpc
  ..Done..  
 ..Processing:  gamma10_rom_mpc
  ..Done..  
 ..Processing:  pad_num_edge_rom_mpc
  ..Done..  
 ..Processing:  pad_num_rom_fp
  ..Done..  


{'XSPx72M8WE': {'type': 'bram_sdp',
  'fpath': 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/ip/XSPx72M8WE/XSPx72M8WE.v',
  'io': {'1st': '    XSPx72M8WE',
   'dout': '        .dout(dout_o), \n',
   'clka': '        .clka(clka_i), \n',
   'cea': '        .cea(cea_i), \n',
   'reseta': '        .reseta(reseta_i), \n',
   'clkb': '        .clkb(clkb_i), \n',
   'ceb': '        .ceb(ceb_i), \n',
   'resetb': '        .resetb(resetb_i), \n',
   'oce': '        .oce(oce_i), \n',
   'ada': '        .ada(ada_i), \n',
   'din': '        .din(din_i), \n',
   'adb': '        .adb(adb_i) \n',
   '-1l': '    );\n'}},
 'dist_32x16m': {'type': 'dram_sdp',
  'fpath': 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/ip/dist_32x16m/dist_32x16m.v',
  'io': {'1st': '    dist_32x16m',
   'dout': '        .dout(dout_o), \n',
   'di': '        .di(di_i), \n',
   'wad': '        .wad(wad_i), \n',
   'rad': '       

In [248]:
x2g_IPs_proc(xip_fps,dst_path)

Gowin Synthesis File: e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/impl\gwsynthesis\min_dual_rcv.prj
 ..Processing:  XSPx72M8WE
 ** XSPx72M8WE: Done **
 ..Processing:  dist_32x16m
 ** dist_32x16m: Done **
 ..Processing:  signed_mult_18x18
 ** signed_mult_18x18: Done **
 ..Processing:  sdp_bram_a8b_d8b
 ** sdp_bram_a8b_d8b: Done **
 ..Processing:  sdp_bram_a10b_d30b
 ** sdp_bram_a10b_d30b: Done **
 ..Processing:  sdp_bram_a10b_d16b
 ** sdp_bram_a10b_d16b: Done **
 ..Processing:  XSP9x512WE
 ** XSP9x512WE: Done **
 ..Processing:  XSP32x1024M4WE
 ** XSP32x1024M4WE: Done **
 ..Processing:  gamma30_rom_mpc
 ** gamma30_rom_mpc: Done **
 ..Processing:  gamma28_rom_mpc
 ** gamma28_rom_mpc: Done **
 ..Processing:  gamma25_rom_mpc
 ** gamma25_rom_mpc: Done **
 ..Processing:  gamma24_rom_mpc
 ** gamma24_rom_mpc: Done **
 ..Processing:  gamma22_rom_mpc
 ** gamma22_rom_mpc: Done **
 ..Processing:  gamma20_rom_mpc
 ** gamma20_rom_mpc: Done **
 ..Proce

{'XSPx72M8WE': {'type': 'bram_sdp',
  'fpath': 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/ip/XSPx72M8WE/XSPx72M8WE.v',
  'io': {'dout': '        .dout(dout_o), \n',
   'clka': '        .clka(clka_i), \n',
   'cea': '        .cea(cea_i), \n',
   'reseta': '        .reseta(reseta_i), \n',
   'clkb': '        .clkb(clkb_i), \n',
   'ceb': '        .ceb(ceb_i), \n',
   'resetb': '        .resetb(resetb_i), \n',
   'oce': '        .oce(oce_i), \n',
   'ada': '        .ada(ada_i), \n',
   'din': '        .din(din_i), \n',
   'adb': '        .adb(adb_i) \n'}},
 'dist_32x16m': {'type': 'dram_sdp',
  'fpath': 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/ip/dist_32x16m/dist_32x16m.v',
  'io': {'dout': '        .dout(dout_o), \n',
   'di': '        .di(di_i), \n',
   'wad': '        .wad(wad_i), \n',
   'rad': '        .rad(rad_i), \n',
   'wre': '        .wre(wre_i), \n',
   'clk': '        .clk

In [20]:
def gw_tmp_extractor(fpath):

    tmp_fpath = fpath + '_tmp.v'
    tmp_rs = {}
    if os.path.exists(tmp_fpath):
        #m = re.compile(r'\.(\w+)\s*\(\s*(\w+)\s*\)')        
        m1 = re.compile(r'\.(\w+)')
        m2 = re.compile(r'//.+')
        ip_name = os.path.split(fpath)[1]
        m3 = re.compile(ip_name)
        for line in open(tmp_fpath,'r'):
            if line.strip() and not line.startswith('//'):
                io = m1.search(line)
                #print(dline,io)
                if io:
                    tmp_rs[io[1]] = m2.sub('',line)
                elif line.strip()[-1] == ';':
                    tmp_rs['-1l'] = line
                else:
                    tmp_rs['1st'] = line[:m3.search(line).span()[1]]
    return tmp_rs

In [21]:
def get_dif_pmv():
    xpmv_path = 'D:/software/Xilinx/14.7/ISE_DS/ISE/vhdl/src/unisims/primitive/'
    XPMVS = [fn.split('.')[0] for fn in os.listdir(xpmv_path)]
    gpmv_path = 'D:/Gowin/Gowin_V1.9.8.05/IDE/bin/primitive.xml'
    data = None
    with open(gpmv_path,'r') as f:
        data = f.read()
    pat = r'<name>(.+?)</name>'
    mch = re.compile(pat)
    GPMVS = mch.findall(data)
    com_pmvs = set(GPMVS) & set(XPMVS)
    dif_pmvs = set(XPMVS) - set(GPMVS)
    #print(com_pmvs)
    #return com_pmvs, dif_pmvs
    return dif_pmvs

In [22]:
get_dif_pmv()

{'CLK_DIV12R',
 'LDC',
 'IBUFGDS_LVDSEXT_33_DCI',
 'OBUFT_LVCMOS25_S_4',
 'DCM',
 'FDR',
 'IDELAYE2_FINEDELAY',
 'OBUFT_LVDCI_15',
 'RAM32X1S',
 'IOBUF_LVCMOS15_F_2',
 'OBUFT_SSTL2_I',
 'OBUF_LVCMOS25_F_6',
 'IBUFGDS_LVDSEXT_25',
 'MULT18X18S',
 'STARTUP_VIRTEX6',
 'IBUF_LVCMOS18',
 'OBUF_LVDCI_DV2_25',
 'XNOR3',
 'OBUFT_GTL_DCI',
 'OBUF_PCIX',
 'DNA_PORT',
 'IBUFG_SSTL18_I_DCI',
 'IOBUFDS_INTERMDISABLE',
 'IBUFDS_GTHE1',
 'OBUFTDS_BLVDS_25',
 'OBUFT_LVCMOS18_F_16',
 'CLK_DIV14RSD',
 'OBUFT_LVCMOS15',
 'OBUFT_HSTL_I_DCI_18',
 'BUFIO2',
 'OBUFT_LVCMOS25_F_2',
 'IOBUF_LVCMOS18_S_8',
 'BUFPLL',
 'IBUFDS_LVPECL_33',
 'IBUF_LVCMOS12',
 'CRC32',
 'OBUFT_LVTTL',
 'FDE',
 'IOBUF_LVCMOS18',
 'OBUFT_LVCMOS12',
 'LUT5_D',
 'OBUF_LVCMOS12_F_2',
 'MMCME2_ADV',
 'IOBUF_S_16',
 'IBUF_SSTL18_II',
 'RAMB16_S2_S36',
 'STARTBUF_VIRTEX',
 'IBUFDS_LVDS_33_DCI',
 'IFDDRCPE',
 'CLK_DIV14',
 'CLK_DIV8RSD',
 'NOR4B2',
 'JTAG_SIM_SPARTAN6',
 'LDCPE_1',
 'IBUF_SSTL3_I',
 'RAM128X1S',
 'IBUF_LVDS',
 'OPT_OFF',
 '

In [24]:
def kick_dnn(dline):
    repl = r'`default_nettype none'
    mch1 = re.compile(repl)  
    #ptn2 = r'`default_nettype none'
    #mch2 = re.compile(ptn2)
    res = mch1.findall(dline)
    line = ''
    if res:
        line = '\n'
        return line
    else:
        return dline

In [23]:
def ff_dnn(fpath):

    inc_dnns = []
    oth_dnns = []
    repl = r'`default_nettype none'
    mch1 = re.compile(repl)  
    ptn = r'^`default_nettype none'
    mch = re.compile(ptn)
    n = 0
    for fn in os.listdir(fpath):
        #data = None
        if fn.split('.')[-1] == 'v':
            fp = os.path.join(fpath,fn)
            for line in open(fp,'r'):
                res = mch.findall(line)
                res1 = mch1.findall(line)
                if res:
                    inc_dnns.append(fn)
                    #data = mch.sub('',data)
                    #n += 1
                elif res1:
                    oth_dnns.append(fn)
                #if n == 1:
                    #print(line)
    return inc_dnns, oth_dnns

In [24]:
#def xlx_freeze(dline, cmt=False, DIF_XPMVS=None):
#def xlx_freeze(dline, cmt=False, xip_mark=False, xpmv_mark=False, io_long=False, DIF_XPMVS=None, ip_names=None, dpath=dst_path):
def xlx_freeze(dline, xip_mark=False, xpmv_mark=False, io_long=False, DIF_XPMVS=None, ip_names=None, dpath=dst_path, update=False):
    
    ptn1 = r'\(\*|\*\)'
    mch1 = re.compile(ptn1)
    ptn2 = r'VCC\s+'
    mch2 = re.compile(ptn2) 
    ptn3 = r'^[A-Z]\w+\s+'
    #ptn3 = r'^\w+\s+'
    mch3 = re.compile(ptn3)
    #mch4 = re.compile(r'\);')
    mch4 = re.compile(r';')
    mch5 = re.compile(r'(\w+)\s*\(')

    line = ''
    ip_name = ''
    pmv = ''
    inst_name = ''
    #xip_mark = False
    io = None
    #xip_blk = False
    #dl = dline.strip()

    if dline.strip():
        #print('Line: ',dline)
        lh = dline.strip().split()[0]
        #lt = dline.strip()[-1]    
        #lt = mch4.search(dline.strip())    
        #lt = dline.strip()[-1] == ';'   
        lt = ';' in dline.strip()   
        if xip_mark or xpmv_mark:
            #print('xip_mark: ',xip_mark)
            #io = xip_entity_extract(dline.strip(),cmt=cmt)
            io, io_long = xip_entity_extract(dline.strip(),xip_mark=xip_mark,xpmv_mark=xpmv_mark,io_long=io_long)
            #if dline.strip()[-1] == ';':
            #if lt == ';':
            if lt:
                xip_mark = False
                xpmv_mark =False
                #cmt = False
            line = '//' + dline
        elif mch1.findall(dline) and not dline.strip().startswith('//'):             #(*   *) comment
            line = '//' + dline     
        elif mch2.findall(dline):             #VCC xxx(.P(xxx)) to VCC xxx(.V(xxx))
            line = dline.replace('.P','.V')  
        #elif mch3.findall(dl):
            #pmv = dl.split()[0]
        elif lh in DIF_XPMVS:                #other primitives process
            #print('Line: ',dline)
            #print('Got XPMV!: {} - XIP_MARK: {}'.format(lh,xip_mark))        
            #cmt = True
            pmv = lh
            dp = os.path.join(dpath,'xprims')
            if not os.path.exists(dp):
                os.makedirs(dp)
            #xpmv_mark = True
            #ip_name = lh
            #io, io_long = xip_entity_extract(dline.strip(),xpmv_mark=xpmv_mark,io_long=io_long)
            if any(['DDR' in pmv, 'PLL' in pmv, 'DCM' in pmv, 'DELAY' in pmv]):
                xpmv_mark = True
                io, io_long = xip_entity_extract(dline.strip(),xpmv_mark=xpmv_mark,io_long=io_long)
                line = '//' + dline
            else:
                pmv_process(pmv,dp,PMVS=DIF_XPMVS,update=update)
                line = dline
        elif lh in ip_names:                #Xilinx IP process
            #print('Line: ',dline)
            #print('Got IP!: {}'.format(lh))        
            #cmt = True
            xip_mark = True
            ip_name = lh
            #inst_name = dline.strip().split()[1].strip('(')
            inst_name = dline.strip().split()[1].split('(')[0].strip()
            #io, io_long = xip_entity_extract(dline.strip(),cmt=cmt,io_long=io_long)
            io, io_long = xip_entity_extract(dline.strip(),xip_mark=xip_mark,io_long=io_long)
            #print('IP Sub: {} - XIP_MARK: {} - IO: {} - IO_LONG: {}'.format(lh, xip_mark, io, io_long))            
            line = '//' + dline            
        else:
            line = dline
    else:
        line = dline            

    #return line, cmt, ip_name, inst_name, io, io_long
    return line, xip_mark, xpmv_mark, pmv, ip_name, inst_name, io, io_long

In [25]:
def pmv_process(pmv,dpath,PMVS=get_dif_pmv(),update=False):
    spath = 'D:/software/Xilinx/14.7/ISE_DS/ISE/verilog/src/unisims/'
    pmv_spath = os.path.join(spath,pmv+'.v')
    if os.path.exists(pmv_spath):
        if pmv in PMVS:
            dat = None
            with open(pmv_spath,'r') as f:
                dat = f.readlines()
            nlines = []
            gsr_vars = []
            cmt = False
            for line in dat:
                nline, cmt, gsr_vars = gsr_ban(line, cmt=cmt, gsr_vars=gsr_vars)
                
                nlines.append(nline)
            pmv_dpath = os.path.join(dpath,pmv+'.v')
            if update:
                with open(pmv_dpath,'w') as f:
                    f.writelines(nlines)
            for line in dat:
                md = line.strip().split(' ')[0]
                pmv_process(md,dpath,PMVS=PMVS)

In [158]:
l = '    always @(GSR)'

In [161]:
gsr_ban(l, gsr_vars=['GSR'])

elm:  ['always', '@(GSR)']
trigs:  ['GSR']
boom:  ['GSR']
Line:     always @(GSR) - CMT: True - GSR: ['GSR']


('//    always @(GSR)', True, ['GSR'])

In [26]:
def gsr_ban(dline, cmt=False, gsr_vars=[]):
    nline = ''
    elms = [e for e in dline.strip().split(' ') if e]
    m = re.compile(r'\((.+)\)')
    trigs = m.findall(dline.strip())
    #specify = True if dline.strip().split(' ')[0] == 'specify' else False
    if elms:
        if 'glbl' in dline:
            cmt = True
            eql_idx = elms.index('=') if '=' in elms else 0
            if eql_idx:
                gsr_var = elms[eql_idx-1]
                gsr_vars.append(gsr_var)
                #print('Line: {} - CMT: {} - GSR: {}'.format(dline,cmt,gsr_vars))        
        elif elms[0] == 'always' and trigs:
            boom = [e for e in trigs[0].split(' ') if e and e in gsr_vars]
            #print('elm: ',elms)
            #print('trigs: ',trigs)
            #print('boom: ',boom)
            if boom:                
                cmt = True
        elif elms[0] == 'specify':
            cmt = True
    elif cmt:
        #print('End')
        cmt = False
    if cmt:
        nline = '//' + dline
        #print('Line: {} - CMT: {} - GSR: {}'.format(dline,cmt,gsr_vars))
    else:
        nline = dline
    #print('Line: {} - CMT: {} - GSR: {}'.format(dline,cmt,gsr_vars))    
    return nline, cmt, gsr_vars

In [27]:
#def xip_entity_extract(dline, cmt=False, io_long=False):
def xip_entity_extract(dline, xip_mark=False, xpmv_mark=False, io_long=False):
    if xip_mark or xpmv_mark:
        if not io_long:
            m1 = re.compile(r'\.(\w+)\s*\(\s*(.+)\s*\)')
            io = m1.findall(dline)
            #print(dline,io)
            if io:
                if xip_mark:
                    return (io[0][0], '(' + io[0][1] + ')' if ')' not in io[0][1] else '(' + io[0][1]), io_long
                elif xpmv_mark:
                    return (io[0][0], io[0][1] if ')' not in io[0][1] else io[0][1].strip(')')), io_long
            else:
                m2 = re.compile(r'\.(\w+)\s*\(\s*(.+)\s*')
                io = m2.findall(dline)
                io_long = True
                if io:
                    if xip_mark:
                        return (io[0][0], '(' + io[0][1]), io_long
                    elif xpmv_mark:
                        return (io[0][0], io[0][1]), io_long
                else:
                    return '', False
        else:
            #print('*Long Port Found*')
            if '),' in dline: 
                io = dline.strip('),')
                #m3 = re.compile(r'(.+)\),')
                #io = m3.findall(dline)
                if io:
                    io_long = False
                    #print('*Long Port Ended*')
                    if xip_mark:
                        return ('',' ' + io + ')'), io_long
                    elif xpmv_mark:
                        return ('',' ' + io), io_long
            else:
                io_long = True
                return ('',dline), io_long
    else:
        return '', False

In [28]:
prims_path = 'D:/software/Xilinx/14.7/ISE_DS/ISE/verilog/'

In [28]:
def search_prims_atom(path, mdl='buf'):
    if os.path.isfile(path):
        print('\r{}'.format(path),end='')
        flines = None
        try:
            with open(path, 'r') as f:
                flines = f.readlines()
        except:
            flines = []
        pearl = False
        res = []
        if flines:
            for line in flines:
                md = [d for d in line.strip().split(' ') if d]
                if md[:2] == ['module', mdl]:
                    pearl = True
                    res.append(line)
                elif pearl:
                    res.append(line)
                    if ';' in line:
                        pearl = False
            if res:
                print()
                print(path, res)
                return path, res
    
    elif os.path.isdir(path):
        elm_paths = [os.path.join(path, elm) for elm in os.listdir(path)]
        for elm in elm_paths:
            search_prims_atom(elm, mdl=mdl)

In [29]:
def xpmv_parse(pmv_info={}, target_family=1):
    nlines = []
    if target_family and pmv_info:
        pmv = pmv_info['pmv']
        if any(['AND' in pmv, 'XOR' in pmv, 'OR' in pmv]):
             nlines.append(op_pmv_proc(pmv_info))
        elif any(['BUF' in pmv, '']):
            pass
    return nlines

In [30]:
def op_pmv_proc(pmv_info):
    pmv = pmv_info['pmv']
    O = pmv_info['O']
    i0 = '~' + pmv_info['I0'] if 'B' in pmv else pmv_info['I0']
    op = op_mark_handle(pmv)
    tail = [op + pmv_info[k] for k in pmv_info if k not in ['O', 'pmv', 'I0']]
    return ''.join(['\t', 'assign ', O, ' = ',  i0] + tail)

In [31]:
def op_mark_handle(pmv):
    if 'AND' in pmv:
        return ' & '
    elif 'XOR' in pmv:
        return ' ^ '
    elif 'OR' in pmv:
        return ' | '    

In [38]:
def spec_fname_rtn(spec, fpaths, PMVS=get_dif_pmv(), xip_fpaths=xip_fps):
    tars = []
    pmv_bio = {}
    for fp in fpaths:
        res = []
        nlines = None
        with open(fp,'r') as f:
            nlines = f.readlines()
        #p = ';\s+(' + spec + '\s+' + '.+;)'
        #p = spec
        #print(p)
        #m = re.compile(p)
        ips = []
        if xip_fpaths:
            ips = [os.path.split(fp)[-1].split('.')[0] for fp in xip_fpaths]
        atv = False
        for line in nlines:  
            x = line.strip().split(' ')[0]
            if x == spec:
                if x in PMVS or x in ips: 
                    atv = True
                    res.append(line)
            elif atv:
                if ';' in line:
                    atv = False
                res.append(line)                
        if res:
            pmv_bio[fp] = res
            if fp not in tars:
                tars.append(fp)            
    return tars, pmv_bio

In [38]:
s = '   XSPx72M8WE   IM0R ('

In [39]:
xip_entity_extract(s.strip(),xip_mark=True)

('', False)

In [33]:
def ip_shift(src_ip_info={}, dst_ips_info={}, target_family=1):
    nlines = []
    if src_ip_info and dst_ips_info:
        ip_name = src_ip_info['ip_name']
        #if target_family == 1:
        #    print('***IP Migration: X2G***\n')
        #print('ip_name: ',ip_name)
        inst_name = src_ip_info['inst_name']
        #print('inst_name: ',inst_name)
        tmp_1st = dst_ips_info[ip_name]['tmp']['1st']
        line_1 = '\t' + ip_name + ' ' + inst_name + '(' + '\n'
        nlines.append(line_1)
        
        if target_family == 1: #Gowin
            m = re.compile(r'\(\w+\)')
            ip_type = dst_ips_info[ip_name]['type']
            if ip_type == 'bram_sdp':
                nlines.append(m.sub(src_ip_info['doutb'],dst_ips_info[ip_name]['tmp']['dout'  ]))
                nlines.append(m.sub(src_ip_info['clka'] ,dst_ips_info[ip_name]['tmp']['clka'  ]))
                nlines.append(m.sub(src_ip_info['wea']  ,dst_ips_info[ip_name]['tmp']['cea'   ]))
                nlines.append(m.sub('()'                ,dst_ips_info[ip_name]['tmp']['reseta']))
                nlines.append(m.sub(src_ip_info['clkb'] ,dst_ips_info[ip_name]['tmp']['clkb'  ]))
                nlines.append(m.sub('(1)'               ,dst_ips_info[ip_name]['tmp']['ceb'   ]))
                nlines.append(m.sub('()'                ,dst_ips_info[ip_name]['tmp']['resetb']))
                nlines.append(m.sub('()'                ,dst_ips_info[ip_name]['tmp']['oce'   ]))
                nlines.append(m.sub(src_ip_info['addra'],dst_ips_info[ip_name]['tmp']['ada'   ]))
                nlines.append(m.sub(src_ip_info['dina'] ,dst_ips_info[ip_name]['tmp']['din'   ]))
                nlines.append(m.sub(src_ip_info['addrb'],dst_ips_info[ip_name]['tmp']['adb'   ]))
                nlines.append(                           dst_ips_info[ip_name]['tmp']['-1l'   ])
            elif ip_type == 'prom': 
                nlines.append(m.sub(src_ip_info['douta'],dst_ips_info[ip_name]['tmp']['dout' ]))
                nlines.append(m.sub(src_ip_info['clka'] ,dst_ips_info[ip_name]['tmp']['clk'  ]))
                nlines.append(m.sub('()'                ,dst_ips_info[ip_name]['tmp']['oce'  ]))
                nlines.append(m.sub(src_ip_info['ena']  ,dst_ips_info[ip_name]['tmp']['ce'   ]))
                nlines.append(m.sub('()'                ,dst_ips_info[ip_name]['tmp']['reset']))
                nlines.append(m.sub(src_ip_info['addra'],dst_ips_info[ip_name]['tmp']['ad'   ]))
                nlines.append(                           dst_ips_info[ip_name]['tmp']['-1l'  ])
            elif ip_type == 'dram_sdp': 
                nlines.append(m.sub(src_ip_info['qdpo'] ,dst_ips_info[ip_name]['tmp']['dout']))
                nlines.append(m.sub(src_ip_info['d']    ,dst_ips_info[ip_name]['tmp']['di'  ]))
                nlines.append(m.sub(src_ip_info['a']    ,dst_ips_info[ip_name]['tmp']['wad' ]))
                nlines.append(m.sub(src_ip_info['dpra'] ,dst_ips_info[ip_name]['tmp']['rad' ]))
                nlines.append(m.sub(src_ip_info['we']   ,dst_ips_info[ip_name]['tmp']['wre' ]))
                nlines.append(m.sub(src_ip_info['clk']  ,dst_ips_info[ip_name]['tmp']['clk' ]))
                nlines.append(                           dst_ips_info[ip_name]['tmp']['-1l' ])               
            elif ip_type == 'sign_mult': 
                nlines.append(m.sub(src_ip_info['p']    ,dst_ips_info[ip_name]['tmp']['dout' ]))
                nlines.append(m.sub(src_ip_info['a']    ,dst_ips_info[ip_name]['tmp']['a'    ]))
                nlines.append(m.sub(src_ip_info['b']    ,dst_ips_info[ip_name]['tmp']['b'    ]))
                nlines.append(m.sub('(1)'               ,dst_ips_info[ip_name]['tmp']['ce'   ]))
                nlines.append(m.sub(src_ip_info['clk']  ,dst_ips_info[ip_name]['tmp']['clk'  ]))
                nlines.append(m.sub('()'                ,dst_ips_info[ip_name]['tmp']['reset']))
                nlines.append(                           dst_ips_info[ip_name]['tmp']['-1l'  ])
    return nlines
        

In [52]:
xip_fps

['e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/ipcore_dir/XSPx72M8WE.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/ipcore_dir/dist_32x16m.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/signed_mult_18x18.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/sdp_bram_a8b_d8b.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/sdp_bram_a10b_d30b.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/sdp_bram_a10b_d16b.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/ipcore_dir/XSP9x512WE.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/ipcore_dir/XSP32x1024M4WE.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_r

In [44]:
prom_fp, dat = spec_fname_rtn('XSPx72M8WE', leaf_paths)

In [61]:
prom_fp, dat = spec_fname_rtn('XSP9x512WE', leaf_paths)

In [39]:
prom_fp, dat = spec_fname_rtn('XSP32x1024M4WE', leaf_paths)

In [69]:
prom_fp, dat = spec_fname_rtn('dist_32x16m', leaf_paths)

In [55]:
prom_fp, dat = spec_fname_rtn('gamma30_rom_mpc', leaf_paths)

In [57]:
prom_fp, dat = spec_fname_rtn('signed_mult_18x18', leaf_paths)

In [63]:
prom_fp, dat = spec_fname_rtn('AND3B1',leaf_paths)

In [64]:
prom_fp, dat = spec_fname_rtn('IBUFG',leaf_paths)

In [59]:
prom_fp, dat = spec_fname_rtn('BUF',leaf_paths)

In [141]:
prom_fp, dat = spec_fname_rtn('PLL_BASE',leaf_paths)

In [144]:
prom_fp, dat = spec_fname_rtn('FD',leaf_paths)

In [40]:
prom_fp

['e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/pti_source/FMEM_BWCNT32M.v']

In [72]:
test1 = 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/gamma_ctemp_fp.v'

In [171]:
rs1, rs2, rs3 = vlgs_calls_proc(prom_fp[0],update=True,dst_ips_info=dst_ips_info)

In [93]:
rs1

[]

In [94]:
rs2

[]

In [95]:
rs3

[]

In [34]:
#def vlg_X2G(spath, dst_dir, dpmvs=get_dif_pmv(), update=True):
def vlgs_calls_proc(spath, dpmvs=get_dif_pmv(), dst_ips_info={}, update=True, target_family=1):
    #if not os.path.exists(dst_dir):
    #    os.makedirs(dst_dir)
    #dpath = os.path.join(dst_dir,os.path.split(spath)[-1])
    nlines = []
    dnn_en = True
    ptm = r'^module'
    mcm = re.compile(ptm)
    #cmt = False
    xip_mark = False
    xpmv_mark = False
    xpmv_proc = False
    io_long = False
    pport = ['','']
    xip_info_book = []
    xip_info = {}
    xpmv_info = {}
    #ip_mn = ''
    xip_proc = False
    lines_proc_book = []
    lines_proc = []
    
    pmvs = []
    
    xpmv_info_book = []
    
    ip_names = dst_ips_info.keys()
    
    for n,dline in enumerate(open(spath,'r')):
    #for dline in open(spath,'rb'):
        if mcm.findall(dline.strip()):
            dnn_en = False
        if dnn_en:
            line = kick_dnn(dline)
            nlines.append(line)
        else:
            #line, cmt, ip_name, inst_name, io = xlx_freeze(dline, cmt=cmt, DIF_XPMVS=dpmvs, ip_names=ip_names)
            line, xip_mark, xpmv_mark, pmv, ip_name, inst_name, io, io_long = xlx_freeze(dline, xip_mark=xip_mark, 
                                                                                              xpmv_mark=xpmv_mark, io_long=io_long, 
                                                                                              DIF_XPMVS=dpmvs, ip_names=ip_names, update=update)           
            nlines.append(line)           
            #if spath not in xpmv_book:
            #    xpmv_book[spath] = [] 
            
            if pmv:              
                #xpmv_book[spath].append(pmv)
                xpmv_info = {}
                xpmv_info['pmv'] = pmv
                pmvs.append(pmv)            
                #print('Line{}: {} -  XIP_MARK: {} - XPMV_MARK: {} - IP: {} - INSTN: {} - IO: {} - IO_LONG: {} - xpmv_info: {} - xip_info: {}'.format(
                #        n+1, line, xip_mark, xpmv_mark,
                #        ip_name, inst_name, 
                #        io, io_long, xpmv_info, xip_info))              
            #lins_proc.append(line)
            #if ip_name and ip_name not in xip_info:
            if ip_name:
                #print('Line{}: {} - XIP_MARK: {} - XPMV_MARK: {} - IP: {} - INSTN: {} - IO: {} - IO_LONG: {} - xpmv_info: {} - xip_info: {}'.format(
                #        n+1, line, xip_mark, xpmv_mark,
                #        ip_name, inst_name, 
                #        io, io_long, xpmv_info, xip_info))
                xip_info = {}
                xip_info['ip_name'] = ip_name
                xip_info['inst_name'] = inst_name
                #if io:
                #if io and not io_long:
                #    xip_info[io[0]] = '(' + io[1] + ')'
            if io:             
                if io[0]:
                    #if io_long:
                    pport = [io[0], io[1]]
                    xip_info[pport[0]] = pport[1]
                    xpmv_info[pport[0]] = pport[1]
                    #print('Line{}: {} - XIP_MARK: {} - XPMV_MARK: {} - IP: {} - INSTN: {} - IO: {} - IO_LONG: {} - xpmv_info: {} - xip_info: {}'.format(
                    #        n+1, line, xip_mark, xpmv_mark,
                    #        ip_name, inst_name, 
                    #        io, io_long, xpmv_info, xip_info))                     
                #elif io_long:
                elif io[1]:
                    p_mline = pport[1] + io[1]
                    xip_info[pport[0]] = p_mline
                    xpmv_info[pport[0]] = p_mline
                    pport[1] = p_mline
                    if not io_long:
                        pport = ['','']
                    #print('Line{}: {} - XIP_MARK: {} - XPMV_MARK: {} - IP: {} - INSTN: {} - IO: {} - IO_LONG: {} - xpmv_info: {} - xip_info: {}'.format(
                    #        n+1, line, cmt, xip_mark, xpmv_mark,
                    #        ip_name, inst_name, 
                    #        io, io_long, xpmv_info, xip_info))            
            if xpmv_mark:
                lines_proc.append(line)
                xpmv_proc = True
            #if xip_proc and not cmt:
            elif xpmv_proc:
                res_lines = xpmv_parse(xpmv_info, target_family=target_family)
                nlines += res_lines
                lines_proc.append(line)
                lines_proc += res_lines
                lines_proc_book.append(lines_proc)
                lines_proc = []
                xpmv_info_book.append(xpmv_info)
                xpmv_info = {}
                xpmv_proc = False  
                
            if xip_mark:
                lines_proc.append(line)
                xip_proc = True
            #if xip_proc and not cmt:
            elif xip_proc:
                res_lines = ip_shift(src_ip_info=xip_info, dst_ips_info=dst_ips_info, target_family=target_family)
                nlines += res_lines
                lines_proc.append(line)
                lines_proc += res_lines
                lines_proc_book.append(lines_proc)
                lines_proc = []
                xip_info_book.append(xip_info)
                xip_info = {}
                xip_proc = False
                
    #if pmvs:
    #    xpmv_info_book[spath] = pmvs
    #with open(dpath,'w') as f:
    if update:
        with open(spath,'w') as f:
        #with open(spath,'wb') as f:
            f.write(''.join(nlines))
    return xip_info_book, lines_proc_book, xpmv_info_book

In [376]:
xip_fps

['e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/ipcore_dir/XSPx72M8WE.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/ipcore_dir/dist_32x16m.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/signed_mult_18x18.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/sdp_bram_a8b_d8b.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/sdp_bram_a10b_d30b.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/sdp_bram_a10b_d16b.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/ipcore_dir/XSP9x512WE.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/ipcore_dir/XSP32x1024M4WE.v',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_r

In [35]:
def prj_proc(spath, dst_ips_info={}, update=False, target_family=1):
    #if update:    
    dpmvs = get_dif_pmv()
    fps = []
    if isinstance(spath,list):
        fps = spath
    elif os.path.isdir(spath):
        fnames = [fn for fn in os.listdir(spath) if os.path.isfile(os.path.join(spath,fn)) and fn.split('.')[1] == 'v']
        #mnames = [fn.split('.')[0] for fn in fnames]
        fps = [os.path.join(spath,fn) for fn in fnames]
    elif os.path.isfile(spath): 
        fps.append(spath)
    err = []
    pmv_book = []
    n = 0
    for i,fp in enumerate(fps):      
        try:
            __, __, ppmv = vlgs_calls_proc(fp, dpmvs=dpmvs, dst_ips_info=dst_ips_info,update=update,target_family=target_family)
            if ppmv:
                pmv_book += ppmv            
            n += 1            
            print('\rProcessed: {}/{}'.format(n,len(fps)),end='')            
        except:
            err.append(fp)
        #__, __, ppmv = vlgs_calls_proc(fp, dpmvs=dpmvs, dst_ips_info=dst_ips_info,update=update,target_family=target_family)
        #if ppmv:
        #    pmv_book.append(ppmv)
        #n += 1
        #print('\rProcessed: {} - {}/{}'.format(os.path.split(fp)[-1],n,len(fps)),end='') 
    print('\nError Case: {}'.format(err))
    return err, pmv_book

In [37]:
dst_path

'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/'

In [36]:
def xprj_reborn(spath,dpath,vlg_update=False,ip_update=False,target_family=1):
    
    mdl_ns = sub_mdl_sp(spath)
    print('Leaves Generating.....')
    xip_fps, leaf_fpaths = fprj_flu(mdl_ns,spath,dpath,update=vlg_update,targe_family=target_family)
    print('*Status: Done - Leaves: {}*'.format(len(leaf_fpaths)))
    dst_ips_info = None
    #gw_ip_names, gw_ip_fpaths = x2g_IPs_proc(xip_fps,dpath)
    if target_family == 1:
        print('IP Generating.....')
        dst_ips_info = x2g_IPs_proc(xip_fps,dpath,update=ip_update)
        print('*Status: Done - IPs: {}/{}*'.format(len(dst_ips_info),len(xip_fps)))
    dst_bad = None
    if target_family != 0:
        print('IP Substituting.....')
        dst_bad, pmv_book = prj_proc(leaf_fpaths,dst_ips_info=dst_ips_info,update=ip_update,target_family=target_family)
        print('*Status: Done - IPs: {}/{}*'.format(len(dst_ips_info),len(xip_fps)))
    return xip_fps, leaf_fpaths, dst_ips_info, dst_bad, pmv_book

In [37]:
xip_fps, leaf_paths, dst_ips_info, dst_bad, pmv_book = xprj_reborn(src_path,dst_path)

XISE:  e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/rcv.xise
PRJ: e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/rcv.prj - Real: True
Leaves Generating.....
*Status: Done - Leaves: 200*
IP Generating.....
 ..Processing:  XSPx72M8WE
  ..Done..  
 ..Processing:  dist_32x16m
  ..Done..  
 ..Processing:  signed_mult_18x18
  ..Done..  
 ..Processing:  sdp_bram_a8b_d8b
  ..Done..  
 ..Processing:  sdp_bram_a10b_d30b
  ..Done..  
 ..Processing:  sdp_bram_a10b_d16b
  ..Done..  
 ..Processing:  XSP9x512WE
  ..Done..  
 ..Processing:  XSP32x1024M4WE
  ..Done..  
 ..Processing:  gamma30_rom_mpc
  ..Done..  
 ..Processing:  gamma28_rom_mpc
  ..Done..  
 ..Processing:  gamma25_rom_mpc
  ..Done..  
 ..Processing:  gamma24_rom_mpc
  ..Done..  
 ..Processing:  gamma22_rom_mpc
  ..Done..  
 ..Processing:  gamma20_rom_mpc
  ..Done..  
 ..Processing:  gamma18_rom_mpc
  ..Done..  
 ..Processing:  gamma10_rom_mpc
  ..Done

In [139]:
xip_fps, leaf_paths, dst_ips_info, dst_bad, pmv_book = xprj_reborn(src_path,dst_path,vlg_update=True)

XISE:  e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/rcv.xise
PRJ: e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/rcv.prj - Real: True
Leaves Generating.....
**VLG: tx_test.v Copyed Done**
**VLG: sens_crc.v Copyed Done**
**VLG: M2_1.v Generated Done**
**VLG: sel8.v Generated Done**
**VLG: rtn_mux_b.v Copyed Done**
**VLG: rgmii_tx.v Generated Done**
**VLG: FD8CE.v Generated Done**
**VLG: fd8.v Generated Done**
**VLG: fd1.v Generated Done**
**VLG: ctrl_data_plus.v Copyed Done**
**VLG: FD8CE.v Generated Done**
**VLG: M2_1.v Generated Done**
**VLG: rgmii_tx.v Generated Done**
**VLG: fd1.v Generated Done**
**VLG: fd8.v Generated Done**
**VLG: sel8.v Generated Done**
**VLG: tx_process.v Generated Done**
**VLG: M2_1.v Generated Done**
**VLG: sel16.v Generated Done**
**VLG: MRCV_PLL.v Copyed Done**
**VLG: MIN_CPNL_RDCNT.v Copyed Done**
**VLG: MIN_CPDT_BUF.v Copyed Done**
**VLG: MIN_BRTC_BF.v Copyed Done**
**VL

In [128]:
pmv_book

[{'pmv': 'ODDR2',
  'DDR_ALIGNMENT': '"C0"), .SRTYPE("ASYNC"), .INIT(1\'b0) ',
  'CE': 'XLXN_4',
  'C0': 'TXC_CLK',
  'C1': 'XLXN_6',
  'D0': 'CLK_OE',
  'D1': 'XLXN_3',
  'R': 'XLXN_3',
  'S': 'XLXN_3',
  'Q': 'TXC'},
 {'pmv': 'ODDR2',
  'DDR_ALIGNMENT': '"C0"), .SRTYPE("ASYNC"), .INIT(1\'b0) ',
  'CE': 'XLXN_11',
  'C0': 'CLK',
  'C1': 'XLXN_12',
  'D0': 'TXD_IN[0]',
  'D1': 'TXD_IN[4]',
  'R': 'XLXN_10',
  'S': 'XLXN_10',
  'Q': 'TXD[0]'},
 {'pmv': 'ODDR2',
  'DDR_ALIGNMENT': '"C0"), .SRTYPE("ASYNC"), .INIT(1\'b0) ',
  'CE': 'XLXN_24',
  'C0': 'CLK',
  'C1': 'XLXN_22',
  'D0': 'TXD_IN[1]',
  'D1': 'TXD_IN[5]',
  'R': 'XLXN_25',
  'S': 'XLXN_25',
  'Q': 'TXD[1]'},
 {'pmv': 'ODDR2',
  'DDR_ALIGNMENT': '"C0"), .SRTYPE("ASYNC"), .INIT(1\'b0) ',
  'CE': 'XLXN_31',
  'C0': 'CLK',
  'C1': 'XLXN_29',
  'D0': 'TXD_IN[2]',
  'D1': 'TXD_IN[6]',
  'R': 'XLXN_32',
  'S': 'XLXN_32',
  'Q': 'TXD[2]'},
 {'pmv': 'ODDR2',
  'DDR_ALIGNMENT': '"C0"), .SRTYPE("ASYNC"), .INIT(1\'b0) ',
  'CE': 'XLXN_38',

In [129]:
len(pmv_book)

69

In [130]:
pmv_book[0]

{'pmv': 'ODDR2',
 'DDR_ALIGNMENT': '"C0"), .SRTYPE("ASYNC"), .INIT(1\'b0) ',
 'CE': 'XLXN_4',
 'C0': 'TXC_CLK',
 'C1': 'XLXN_6',
 'D0': 'CLK_OE',
 'D1': 'XLXN_3',
 'R': 'XLXN_3',
 'S': 'XLXN_3',
 'Q': 'TXC'}

In [133]:
pmvs_gold = set()
for p in pmv_book:
    pmvs_gold.add(p['pmv'])

In [134]:
pmvs_gold

{'DCM_CLKGEN', 'IDDR2', 'IODELAY2', 'ODDR2', 'PLL_BASE'}

In [174]:
xip_fps, leaf_paths, dst_ips_info, dst_bad, pmv_book = xprj_reborn(src_path,dst_path,vlg_update=True,ip_update=True)

XISE:  e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/rcv.xise
PRJ: e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/rcv.prj - Real: True
Leaves Generating.....
**VLG: tx_test.v Copyed Done**
**VLG: sens_crc.v Copyed Done**
**VLG: M2_1.v Generated Done**
**VLG: sel8.v Generated Done**
**VLG: rtn_mux_b.v Copyed Done**
**VLG: rgmii_tx.v Generated Done**
**VLG: FD8CE.v Generated Done**
**VLG: fd8.v Generated Done**
**VLG: fd1.v Generated Done**
**VLG: ctrl_data_plus.v Copyed Done**
**VLG: FD8CE.v Generated Done**
**VLG: M2_1.v Generated Done**
**VLG: rgmii_tx.v Generated Done**
**VLG: fd1.v Generated Done**
**VLG: fd8.v Generated Done**
**VLG: sel8.v Generated Done**
**VLG: tx_process.v Generated Done**
**VLG: M2_1.v Generated Done**
**VLG: sel16.v Generated Done**
**VLG: MRCV_PLL.v Copyed Done**
**VLG: MIN_CPNL_RDCNT.v Copyed Done**
**VLG: MIN_CPDT_BUF.v Copyed Done**
**VLG: MIN_BRTC_BF.v Copyed Done**
**VL

 ** gamma20_rom_mpc: Done **
  ..Done..  
 ..Processing:  gamma18_rom_mpc
 ** gamma18_rom_mpc: Done **
  ..Done..  
 ..Processing:  gamma10_rom_mpc
 ** gamma10_rom_mpc: Done **
  ..Done..  
 ..Processing:  pad_num_edge_rom_mpc
 ** pad_num_edge_rom_mpc: Done **
  ..Done..  
 ..Processing:  pad_num_rom_fp
 ** pad_num_rom_fp: Done **
  ..Done..  
*Status: Done - IPs: 18/18*
IP Substituting.....
Processed: 195/200
Error Case: ['e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/gamma_ctemp_fp.v', 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/m4_mult_xt2p5_a.v', 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/gamma_ctemp_fp.v', 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/m4_mult_xt2p5_a.v', 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/video_process_fp.v']

In [331]:
dst_ips_info['XSPx72M8WE']

{'type': 'bram_sdp',
 'fpath': 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/ip/XSPx72M8WE/XSPx72M8WE.v',
 'io': {'1st': '    XSPx72M8WE',
  'dout': '        .dout(dout_o), \n',
  'clka': '        .clka(clka_i), \n',
  'cea': '        .cea(cea_i), \n',
  'reseta': '        .reseta(reseta_i), \n',
  'clkb': '        .clkb(clkb_i), \n',
  'ceb': '        .ceb(ceb_i), \n',
  'resetb': '        .resetb(resetb_i), \n',
  'oce': '        .oce(oce_i), \n',
  'ada': '        .ada(ada_i), \n',
  'din': '        .din(din_i), \n',
  'adb': '        .adb(adb_i) \n',
  '-1l': '    );\n'}}

In [92]:
#len(xip_fps),xip_fps

In [93]:
#len(gw_ip_names),gw_ip_names

In [94]:
#len(gw_ip_fpaths),gw_ip_fpaths

In [86]:
len(leaf_paths)

200

In [96]:
'PLL_BASE' in get_dif_pmv()

True

In [104]:
'BUFG' in get_dif_pmv()

False

In [743]:
'FDCE' in get_dif_pmv()

True

In [148]:
gw_ip_names[4]

'sdp_bram_a10b_d30b'

In [818]:
ip_fname_rtn(gw_ip_names[4], leaf_paths)

sdp_bram_a10b_d30b ;.+(sdp_bram_a10b_d30b\s+.+?;) e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/video_input_fp_a.v


('e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/video_input_fp_a.v',
 'sdp_bram_a10b_d30b  sdp_bram_a10b_d30b_0(\n      .dina   (lm_din),   // 30bit\n      .wea    (lm_we),    // 1bit\n      .addra  (lm_wa),    // 10bit\n      .clka   (clk),      // Write clock : clk\n      .doutb  (lm_dout),  // 30bit\n      .addrb  (lm_ra),    // 10bit\n      .clkb   (oclk)      // Read clock : oclk\n    );')

In [158]:
pure_path = 'E:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/mini_rcv/dual/Mini_rcv_CPNdual_C_All/pti_source/MRCV_PLL.v'
#thief_fpath = 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/gowin/nightly/mini_rcv/dual/Mini_Rcv_CPNdual_20220718/pti_source/MRCV_PLL.v'
#thief_fpath, dat = ip_fname_rtn(gw_ip_names[4], leaf_paths)
thief_fpath, dat = ip_fname_rtn(gw_ip_names[4], leaf_paths)

In [91]:
thief_fpath in leaf_paths

True

In [125]:
def rewrite_vlg(spath,dpath):
    with open(spath,'r') as f:
        data = f.read()
        with open(dpath,'w') as ff:
            ff.write(data)

In [130]:
rewrite_vlg(pure_path,thief_fpath)

In [5]:
import numpy as np

In [6]:
def read_strategy(path):
    m0 = re.compile(r':(.+?)".+?value\s*=\s*"(.+?)"',re.DOTALL) 
    #m0 = re.compile(r':(.+?)".+?') 
    data = None
    with open(path,'r') as f:
        data = f.read()
    #pairs = np.array(m0.findall(data))
    pairs = m0.findall(data)
    seqs = {entry: value for entry, value in pairs}
    #return pairs    
    return seqs

In [7]:
def get_xds_paths(fdir):
    fnames = os.listdir(fdir)
    xds_fns = [fn for fn in fnames if fn.split('.')[-1] == 'xds']
    xds_fps = [os.path.join(fdir,xds_fn) for xds_fn in xds_fns]
    return xds_fps

In [8]:
get_xds_paths(dst_path)

['e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\currentprojectnavigatorsettingsct84.xds',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\currentprojectnavigatorsettingsct93.xds',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\mapextraeffortioreg.xds',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\mapextraeffortioregct14.xds',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\mapextraeffortioregct14_old.xds',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\mapextraeffortioregct22.xds',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\mapextraeffortioregct22_old.xds',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\mapextraeffortioreg_old.xds',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\maplogoptregdup.xds',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\

In [11]:
e0 = read_strategy(get_xds_paths(dst_path)[5])

In [12]:
e0

{'Placer Effort Level': 'High',
 'Placer Extra Effort': 'Normal',
 'Pack I/O Registers/Latches into IOBs': 'For Inputs and Outputs',
 'Starting Placer Cost Table (1-100)': '22',
 'Place &amp; Route Effort Level (Overall)': 'High',
 'Extra Effort (Highest PAR level only)': 'Normal',
 'Number of Paths in Error/Verbose Report': '3',
 'Report Type': 'Verbose Report',
 'Change Device Speed To': '-2',
 'Report Paths by Endpoint': '3',
 'Report Fastest Path(s) in Each Constraint': 'true'}

In [13]:
len(e0)

11

In [14]:
import pandas as pd

In [288]:
pd.DataFrame??

In [ ]:
def xplorer_show(fdir):
    xp_fps = os.path.join(fdir,'smartxplorer.xml')
    m0 = re.compile('')
    xds_fps = find_xds(fdir)
    xds_pairs = []
    row_idx = None
    col_idx = []
    for i,xds_fp in enumerate(xds_fps):
        stg = read_strategy(xds_fp)
        col_idx.append()
        if i == 0:
            row_idx = list(stg[:,0])
        xds_pairs.append(stg[:,1])
    

In [15]:
import xml.etree.cElementTree as xeT

In [16]:
xp_fps = os.path.join(dst_path,'smartxplorer_results/smartxplorer.xml')

In [17]:
xp_dir = os.path.join(dst_path,'smartxplorer_results')

In [18]:
xp_fps

'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\smartxplorer_results/smartxplorer.xml'

In [19]:
xml_dat = None
with open(xp_fps,'r') as f:
    xml_dat = f.read()

In [20]:
len(xml_dat)

7609

In [23]:
def catch_MAP_COST(path):
    m0 = re.compile(r'Starting Placer Cost Table \(1-100\)" value="(.+?)"') 
    #m0 = re.compile(r'Starting Placer Cost Table \(1-100\)"') 
    #m0 = re.compile(r':(.+?)".+?') 
    data = None
    with open(path,'r') as f:
        data = f.read()
    #pairs = np.array(m0.findall(data))
    cost = m0.findall(data)
    #seqs = {entry: value for entry, value in pairs}
    #return pairs    
    return int(cost[0])

In [26]:
xds_fps[0]

'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\smartxplorer_results\\run1/maproutabilityparstandard1.xds'

In [27]:
catch_MAP_COST(xds_fps[0])

IndexError: list index out of range

In [24]:
from matplotlib import pyplot as plt
from matplotlib import figure as fig

In [351]:
#def mod_fast_iter(context, func, *args, **kwargs):
#    for event,elm in context:
#        #if event == 'end':
#        if elm.tag == 'programStatus':
#            if elm.text == 'Running':
#                running = True
#            else:
#                running = False
#            #print(elm.text)
#        if elm.tag == 'strategy':
#            strategy.append(elm)
#        elif elm.tag == 'xdsFile':
#            xdsFiles.append(elm.text.strip())
#        elif elm.tag == 'runIndex':
#            runidxs.append(elm.text.strip())
#        elif elm.tag == 'status':
#            status.append(elm.text.strip()) 
#        elif elm.tag == 'timingScore':
#            timingS.append(elm.text.strip())
#        elif elm.tag == 'WorstCaseSlack':
#            worstCaseSlack.append(elm.text.strip())
#        elif elm.tag == 'Luts':
#            luts.append(elm.text.strip())
#        elif elm.tag == 'SliceRegisters':
#            sliceRegisters.append(elm.text.strip())
#        elif elm.tag == 'runTime':
#            runTime.append(elm.text.strip())
#        #print(elm.tag)
#        elm.clear()
#    #break 
#        for ancestor in elm.xpath('ancestor-or-self::*'):
#            while ancestor.getprevious() is not None:
#                del ancestor.getparent()[0]
#    del context
#
#def func(elem):
#    content = '' if not elem.text else elem.text.strip()
#    if content:
#        ancestors = elem.xpath('ancestor-or-self::*')
#        print ('%s=%s' % ('.'.join([_.tag for _ in ancestors]), content))

In [ ]:
xeT.i

In [25]:
strategy = []
runidxs = []
xdsFiles = []
status = []
timingS = []
worstCaseSlack = []
luts = []
sliceRegisters = []
runTime = []
running = False

col_idxs = ['MAP_Cost', 'Status', 'Runtime', 'TimingScore', 'WorstCaseSlack', 'Luts', 'SLRegisters']

xds_rps = []
xds_fps = []
map_cost = []


context = xeT.iterparse(open(xp_fps,'rb'))

for event,elm in context:
    #if event == 'end':
    if elm.tag == 'programStatus':
        if elm.text == 'Running':
            running = True
        else:
            running = False
        #print(elm.text)
    if elm.tag == 'strategy':
        strategy.append(elm)
    elif elm.tag == 'xdsFile':
        xdsFiles.append(elm.text.strip())
    elif elm.tag == 'runIndex':
        runidxs.append(elm.text.strip())
    elif elm.tag == 'status':
        status.append(elm.text.strip()) 
    elif elm.tag == 'timingScore':
        timingS.append(elm.text.strip())
    elif elm.tag == 'WorstCaseSlack':
        worstCaseSlack.append(elm.text.strip())
    elif elm.tag == 'Luts':
        luts.append(elm.text.strip())
    elif elm.tag == 'SliceRegisters':
        sliceRegisters.append(elm.text.strip())
    elif elm.tag == 'runTime':
        runTime.append(elm.text.strip())
    #print(elm.tag)
    elm.clear()
    #break 

del context
    
l = 0
for i,d in enumerate(timingS): 
    l = i + 1
    if d == 'None':
        l = i
        break

runidxs = runidxs[:l]
xdsFiles = xdsFiles[:l]
status  = status[:l]
timingS = [int(x) for x in timingS[:l]]
worstCaseSlack = worstCaseSlack[:l]
luts = luts[:l]
sliceRegisters = sliceRegisters[:l]
runTime = runTime[:l]

print(status[:10],timingS[:10])

xds_rps = [runidxs[i]+'/'+xdsFiles[i] for i in range(len(runidxs))]
xds_fps = [os.path.join(xp_dir,xds_rp) for xds_rp in xds_rps]



for xds_fp in xds_fps:
    map_cost.append(catch_MAP_COST(xds_fp))

tss_idxs = [x[0] for x in sorted([(i,timingS[i]) for i in range(len(timingS))], key=lambda x:x[1], reverse=False)]

print(map_cost[:10],timingS[:10])

fig_size = (20,30)
plt.figure(figsize=fig_size)
#fig1.plot([timingS[x] for x in tss_idxs], [map_cost[x] for x in tss_idxs],'r.')
    #rt_data = np.concatenate()
plt.xlim((0, 100))
plt.ylim((0, 500))    
plt.plot(map_cost,timingS,'r*')
#plt.plot(timingS,'r*')
#plt.plot([map_cost[x] for x in tss_idxs], [timingS[x] for x in tss_idxs], 'r.')
    #rt_data = np.concatenate()

['Routing', 'Routing'] [1231659, 241684]


IndexError: list index out of range

In [315]:
plt.figure?

In [324]:
plt.plot??

In [292]:
rtd = np.array([map_cost,status,runTime,timingS,worstCaseSlack,luts,sliceRegisters])

In [293]:
x, y = rtd.shape

In [294]:
rtd.reshape((y,x)).shape

(62, 7)

In [303]:
%matplotlib inline

In [295]:
sorted??

In [257]:
strategy[0].attrib

{}

In [258]:
xdsFiles

['currentprojectnavigatorsettingsct1.xds',
 'currentprojectnavigatorsettingsct2.xds',
 'currentprojectnavigatorsettingsct3.xds',
 'currentprojectnavigatorsettingsct4.xds',
 'currentprojectnavigatorsettingsct5.xds',
 'currentprojectnavigatorsettingsct6.xds',
 'currentprojectnavigatorsettingsct7.xds',
 'currentprojectnavigatorsettingsct8.xds',
 'currentprojectnavigatorsettingsct9.xds',
 'currentprojectnavigatorsettingsct10.xds',
 'currentprojectnavigatorsettingsct11.xds',
 'currentprojectnavigatorsettingsct12.xds',
 'currentprojectnavigatorsettingsct13.xds',
 'currentprojectnavigatorsettingsct14.xds',
 'currentprojectnavigatorsettingsct15.xds',
 'currentprojectnavigatorsettingsct16.xds',
 'currentprojectnavigatorsettingsct17.xds',
 'currentprojectnavigatorsettingsct18.xds',
 'currentprojectnavigatorsettingsct19.xds',
 'currentprojectnavigatorsettingsct20.xds',
 'currentprojectnavigatorsettingsct21.xds',
 'currentprojectnavigatorsettingsct22.xds',
 'currentprojectnavigatorsettingsct23.xds

In [259]:
runidxs

['run1',
 'run2',
 'run3',
 'run4',
 'run5',
 'run6',
 'run7',
 'run8',
 'run9',
 'run10',
 'run11',
 'run12',
 'run13',
 'run14',
 'run15',
 'run16',
 'run17',
 'run18',
 'run19',
 'run20',
 'run21',
 'run22',
 'run23',
 'run24',
 'run25',
 'run26',
 'run27',
 'run28',
 'run29',
 'run30',
 'run31',
 'run32',
 'run33',
 'run34',
 'run35',
 'run36',
 'run37',
 'run38',
 'run39',
 'run40',
 'run41',
 'run42',
 'run43',
 'run44',
 'run45',
 'run46']

In [330]:
runTime

['0h 46m 56s',
 '0h 49m 15s',
 '0h 41m 49s',
 '0h 45m 4s',
 '0h 45m 35s',
 '0h 45m 52s',
 '0h 35m 52s',
 '0h 42m 11s',
 '0h 42m 7s',
 '0h 40m 50s',
 '1h 1m 24s',
 '0h 46m 33s',
 '1h 16m 53s',
 '0h 34m 54s',
 '0h 40m 18s',
 '0h 38m 15s',
 '0h 59m 11s',
 '0h 42m 11s',
 '0h 34m 1s',
 '0h 52m 55s',
 '0h 46m 35s',
 '0h 30m 30s',
 '0h 49m 11s',
 '0h 33m 10s',
 '0h 38m 12s',
 '0h 34m 4s',
 '0h 33m 6s',
 '0h 39m 41s',
 '0h 44m 3s',
 '0h 51m 11s',
 '0h 30m 22s',
 '0h 41m 11s',
 '1h 36m 50s',
 '0h 35m 37s',
 '1h 28m 11s',
 '0h 38m 33s',
 '0h 36m 26s',
 '0h 41m 30s',
 '0h 30m 48s',
 '0h 31m 30s',
 '0h 52m 13s',
 '0h 41m 52s',
 '0h 35m 46s',
 '0h 53m 35s',
 '0h 44m 33s',
 '0h 48m 47s',
 '0h 46m 22s',
 '0h 54m 59s',
 '0h 37m 54s',
 '0h 30m 29s',
 '0h 45m 22s',
 '0h 38m 12s',
 '0h 33m 47s',
 '0h 33m 24s',
 '0h 40m 27s',
 '0h 46m 8s',
 '0h 51m 20s',
 '0h 37m 19s',
 '0h 31m 8s',
 '0h 37m 43s',
 '0h 57m 50s',
 '0h 52m 55s',
 '0h 33m 58s',
 '0h 45m 10s',
 '0h 46m 22s',
 '0h 34m 27s',
 '0h 36m 34s',
 '0h

In [262]:
xds_rps

['run1/currentprojectnavigatorsettingsct1.xds',
 'run2/currentprojectnavigatorsettingsct2.xds',
 'run3/currentprojectnavigatorsettingsct3.xds',
 'run4/currentprojectnavigatorsettingsct4.xds',
 'run5/currentprojectnavigatorsettingsct5.xds',
 'run6/currentprojectnavigatorsettingsct6.xds',
 'run7/currentprojectnavigatorsettingsct7.xds',
 'run8/currentprojectnavigatorsettingsct8.xds',
 'run9/currentprojectnavigatorsettingsct9.xds',
 'run10/currentprojectnavigatorsettingsct10.xds',
 'run11/currentprojectnavigatorsettingsct11.xds',
 'run12/currentprojectnavigatorsettingsct12.xds',
 'run13/currentprojectnavigatorsettingsct13.xds',
 'run14/currentprojectnavigatorsettingsct14.xds',
 'run15/currentprojectnavigatorsettingsct15.xds',
 'run16/currentprojectnavigatorsettingsct16.xds',
 'run17/currentprojectnavigatorsettingsct17.xds',
 'run18/currentprojectnavigatorsettingsct18.xds',
 'run19/currentprojectnavigatorsettingsct19.xds',
 'run20/currentprojectnavigatorsettingsct20.xds',
 'run21/currentpro

In [265]:
xds_fps

['e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\smartxplorer_results\\run1/currentprojectnavigatorsettingsct1.xds',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\smartxplorer_results\\run2/currentprojectnavigatorsettingsct2.xds',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\smartxplorer_results\\run3/currentprojectnavigatorsettingsct3.xds',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\smartxplorer_results\\run4/currentprojectnavigatorsettingsct4.xds',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\smartxplorer_results\\run5/currentprojectnavigatorsettingsct5.xds',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\smartxplorer_results\\run6/currentprojectnavigatorsettingsct6.xds',
 'e:/proj_Enoch/leaps_local/prj_ll/hdl_ll/ise/nightly/Tx_V28_20220325\\smartxplorer_results\\run7/currentprojectnavigatorsettingsct7.xds',
 'e:/proj_Enoch/leaps_local

In [329]:
timingS

['129672',
 '156486',
 '74957',
 '121961',
 '146371',
 '91160',
 '2698',
 '12127',
 '246133',
 '107224',
 '27246',
 '9125',
 '27890',
 '550',
 '24921',
 '107186',
 '16410',
 '5105',
 '91893',
 '56301',
 '63630',
 '300',
 '31688',
 '46576',
 '47634',
 '2012',
 '12987',
 '68971',
 '28952',
 '68783',
 '5974',
 '3340',
 '39179',
 '15643',
 '148923',
 '53367',
 '47978',
 '19448',
 '4404',
 '18701',
 '107658',
 '47941',
 '8322',
 '177725',
 '43397',
 '94885',
 '129132',
 '21816',
 '13889',
 '9077',
 '40415',
 '137629',
 '2586',
 '13880',
 '16047',
 '53959',
 '5814',
 '30145',
 '6407',
 '54834',
 '103139',
 '33831',
 '30404',
 '13914',
 '128765',
 '37715',
 '37514',
 '75728',
 '144806',
 '83159',
 '107073',
 '163602',
 '207317',
 '161958',
 '507943',
 '409238',
 '113287',
 '54460',
 '245948',
 '327371',
 'None',
 'None',
 'None',
 'None',
 'None']

In [222]:
xeT.iterparse??